## 开始

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My\ Drive/Colab Notebooks/OursRepository/public-opinion-monitor

!pip install thulac
!pip install jieba

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/OursRepository/public-opinion-monitor
     |████████████████████████████████| 52.9MB 58kB/s 
  Created wheel for thulac: filename=thulac-0.2.1-cp36-none-any.whl size=53141671 sha256=def17a12758b3354350a38b19039e981a23e982f8de8148abe20362a162623f4
  Stored in directory: /root/.cache/pip/wheels/db/36/4a/1ac1e9b9ce727a9dfc7fa20092992707d7da162df871c8488f
Successfully built thulac


In [21]:
# from ClassicalHANModel import *
# from HANModel import *
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.optimizer import Optimizer
import numpy
import time, math
import torch.utils.data as data
import os
import pandas as pd
import json

import jieba
import thulac
thulacObj = thulac.thulac(seg_only=True)
class thulac_cutor:
    def cut(self,sentence:str):
        return thulacObj.fast_cut(sentence, text=True)
thulac_cutor = thulac_cutor()


Model loaded succeed


## 读取词向量
建立词语列表

In [3]:
from gensim.models import KeyedVectors, Word2Vec

file = '../../DataSets/Word2Vect/Tencent_AILab_ChineseEmbedding/Tencent_AILab_ChineseEmbedding_Min.txt'
# word2vec = Word2Vec.load(file)
word2vec = KeyedVectors.load_word2vec_format(file, binary=False,limit=100000)
# word2vec = KeyedVectors.load_word2vec_format(file, binary=False)
word2vec.init_sims(replace=True)  # 神奇，很省内存，可以运算most_similar
word2vec.vector_size

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


200

In [4]:
print(word2vec)
print(len(word2vec.wv.index2word))
print(word2vec.wv.index2word[0])
print(word2vec.wv.index2word[1])
print(word2vec.wv.index2word[2])
print(word2vec.wv.index2word[1522])
print(word2vec.wv.index2entity[1522])
print(word2vec.similar_by_word('中国'))
print(word2vec.similar_by_word('天才'))
print(word2vec.wv)
print('word2vec.wv.vocab ---- >', word2vec.wv.vocab)
print(word2vec.wv.index2word)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launch

In [5]:
wordEmbedding = [word2vec.wv[word]  for word in word2vec.wv.index2word]
word2index = { word:i for i, word in enumerate(word2vec.wv.index2word)}
# print(wordEmbedding[:2])
# print(word2index['中国'])
# print(word2index['天才'])
print(word2index['喜欢'])
print(word2index['不'])
print(word2index['爽'])
print(word2index['超赞'])



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


217
64
5313
41047


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


## 读取训练数据

In [0]:
# def compute_ngrams(word, num_min = 1, num_max = 3):
#     ngrams =[]
#     for ngram_length in range(num_min, min(len(word), num_max) + 1):
#         for i in range(len(word) - ngram_length + 1):
#             # print(i, i + ngram_length)
#             ngrams.append(word[i : i + ngram_length])
#     # print(ngrams)
#     return list(set(ngrams))

# print(compute_ngrams('you'))
# print(compute_ngrams('I Think'))
# print(compute_ngrams('中华人民共和国万岁'))

In [0]:
# # 从词向量文本文件 word2vec 中获取词向量，如果获取到直接返回，若没有获取到，那么把这个词拆开
# # 成为 ngrams 的新词组，并在 word2vec 中找新词组中的词向量并相加取平均，最后得到平均词向量输出
# def wordVec(word, word2vec, min_n = 1, max_n = 3):
#     # 确认词向量维度
#     word_size = word2vec.wv.syn0[0].shape[0]

#     # 如果在词典之中，直接返回词向量
#     if word in word2vec.wv.vocab.keys():
#         return word2vec[word]
#     else:
#         # 计算word的ngrams词组
#         ngrams = compute_ngrams(word, min_n, max_n)
#         # 不在词典的情况下
#         word_vec = numpy.zeros(word_size, dtype=numpy.float32)
#         ngrams_found = 0
#         ngrams_single = [ng for ng in ngrams if len(ng) == 1]
#         ngrams_more = [ng for ng in ngrams if len(ng) > 1]
#         # 先只接受2个单词长度以上的词向量
#         for ngram in ngrams_more:
#             if ngram in word2vec.wv.vocab.keys():
#                 word_vec += word2vec[ngram]
#                 ngrams_found += 1
#                 #print(ngram)
#         # 如果，没有匹配到，那么最后是考虑单个词向量
#         if ngrams_found == 0:
#             for ngram in ngrams_single:
#                 word_vec += word2vec[ngram]
#                 ngrams_found += 1
#         if word_vec.any():
#             return word_vec / max(1, ngrams_found)
#         else:
#             # 不抛出异常，而是打印提示，并返回0向量。
#             print(KeyError('all ngrams for word %s absent from model' % word))
#             return word_vec

In [0]:
class DotDict(dict):
    def __init__(self, *args, **kwargs):
        dict.__init__(self, *args, **kwargs)
        self.__dict__ = self
        

In [9]:
import pandas as pd

dic = {'a':[1, 2, 3, 4], 'b':[5, 6, 7, 8],
'c':[9, 10, 11, 12], 'd':[13, 14, 15, 16]}
df1=pd.DataFrame(dic)
print(df1)
df2=df1.sample(frac=0.75)
print(df2)
# rowlist=[]
# for indexs in df2.index:
#     rowlist.append(indexs)
df3=df1.drop(df2.index.to_list(),axis=0)
print(df3)

   a  b   c   d
0  1  5   9  13
1  2  6  10  14
2  3  7  11  15
3  4  8  12  16
   a  b   c   d
3  4  8  12  16
1  2  6  10  14
2  3  7  11  15
   a  b  c   d
0  1  5  9  13


In [0]:
# import re

# def splitText(text:str, splitChar = '(。|，|,|！|\!|？|\?|\n|\t)'):
#   '''
#   句子切分分隔符
#   '''
#   contents = re.split(splitChar, text)
#   contents = ["".join([a, b]) if b != '\n' and b != '\t' else "" #a + "。"
#               for a, b in zip(contents[0::2], contents[1::2])]
#   contents = [content for content in contents if content.strip() != '' and content[0] != '。']
#   # for i, sen in enumerate(contents):
#   #     print(i, sen)
#   return contents

# testStr = """
# 深圳，简称“深”，别称鹏城，是广东省副省级市、计划单列市、超大城市，国务院批复确定的中国经济特区、全国性经济中心城市和国际化城市 [1]  。截至2018年末，全市下辖9个区，总面积1997.47平方千米，建成区面积927.96平方千米，常住人口1302.66万人，城镇人口1302.66万人，城镇化率100%，是中国第一个全部城镇化的城市。 [2-5] 
# 深圳地处中国华南地区、广东南部、珠江口东岸，东临大亚湾和大鹏湾，西濒珠江口和伶仃洋，南隔深圳河与香港相连，是粤港澳大湾区四大中心城市之一 [6]  、国家物流枢纽、国际性综合交通枢纽 [7]  、国际科技产业创新中心 [8]  、中国三大全国性金融中心之一 [9-10]  ，并全力建设中国特色社会主义先行示范区 [11]  、综合性国家科学中心 [12]  、全球海洋中心城市 [13]  。深圳水陆空铁口岸俱全，是中国拥有口岸数量最多、出入境人员最多、车流量最大的口岸城市。 [14]
# """

# print(splitText(testStr))

In [0]:
def isNan(a):
    return a != a

class RatingData(data.Dataset):
    def __init__(self, path, 
                 word2index, 
                 max_row = -1, 
                 trainTestRate = 0.85, 
                 isTrain = True, 
                 wordCuter = thulac,
                 dataFile = 'balance_data.csv',
                 clean_file_name = 'ratings_clean_4_HAN.csv',
                 ):
        self.token_list = []
        self.label_list = []

        print(' balance_data.csv 所在path:',path) # 地址不应该包含 ratings.csv

        ratings_clean_filename = os.path.join(path, clean_file_name)
        ratings_filename = os.path.join(path, dataFile)
        if os.path.isfile(ratings_clean_filename):
            clean_pd = pd.read_csv(ratings_clean_filename)
        else:
            print('没有找到缓存的文件%s, 读取源文件%s'%(ratings_clean_filename, ratings_filename))
            ratings_pd = pd.read_csv(ratings_filename)
            print('开始生成缓存文件%s'%(ratings_clean_filename))
            clean_pd = pd.DataFrame({
                'labels':[],
                'data':[],
            })
            nonRatingCount = 0
            for i, row in ratings_pd.iterrows():
                if max_row != -1 and i > max_row:
                    break
                if not isinstance(row['data'], str) or row['data'] == '':
                    # print(i + 1, row['comment'])
                    nonRatingCount += 1
                    continue
                r0 = row['labels']
                if r0 == -1:
                    r0 = 2
                    
                if i % 10000 == 9999:
                    print(i + 1, r0)

                words = list(wordCuter.cut(row['data']))
                # 0 在词向量集中是‘，’，换个词向量集可能表示其他
                token = [ word2index[words[i]] if i < len(words) and words[i] in word2index else 0 
                          for i in range(100)] 
                
                newRow = DotDict()

                newRow.labels = [r0]
                newRow.data = [json.dumps(token)]

                clean_pd = clean_pd.append(pd.DataFrame(newRow), ignore_index=True)
            print('空的评论数量： %d'%(nonRatingCount))
            clean_pd.to_csv(ratings_clean_filename)

        # 读取
        if isTrain:
            temp_pd = clean_pd.sample(frac=trainTestRate)
        else:
            temp_pd = clean_pd.sample(frac=trainTestRate)
            temp_pd = clean_pd.drop(temp_pd.index.tolist(), axis=0)

        for i, row in temp_pd.iterrows():
            if max_row != -1 and i > max_row:
                break

            self.label_list.append(torch.tensor(row['labels']).long())
            self.token_list.append(torch.from_numpy(numpy.array( json.loads(row['data']) ) ).long())

    def __getitem__(self, index):
        return self.token_list[index], self.label_list[index]#, self.token_positions

    def __len__(self):
        return len(self.label_list)
    


In [12]:

ratingData = RatingData('../../DataSets/yf_dianping',
                             word2index = word2index,
                            #  max_row= 5,
                             isTrain=True,
                             trainTestRate = 0.8,
                             wordCuter = jieba,
                             dataFile = 'balance_data.csv',
                             clean_file_name='balance_data_4_HAN_TWV.csv',
                             )
trainLoader = torch.utils.data.DataLoader(dataset=ratingData,
                                          batch_size=512,
                                          shuffle = True,
                                          num_workers = 8,
                                          )
ratingData2 = RatingData('../../DataSets/yf_dianping',
                             word2index = word2index,
                             # max_row= 200000,
                             isTrain=False,
                             trainTestRate = 0.8,
                             wordCuter = jieba,
                             dataFile = 'balance_data.csv',
                             clean_file_name='balance_data_4_HAN_TWV.csv',
                             )
testLoader = torch.utils.data.DataLoader(dataset=ratingData2,
                                          batch_size=512,
                                          shuffle = True,
                                          num_workers = 8,
                                          )
print(len(ratingData.label_list))
print(len(ratingData2.label_list))

 balance_data.csv 所在path: ../../DataSets/yf_dianping
 balance_data.csv 所在path: ../../DataSets/yf_dianping
530311
132578


## 创建模型

In [0]:
isPrint = False

In [0]:
# class SelfAttention(nn.Module):
#     def __init__(self, num_input):
#         super(SelfAttention, self).__init__()
#         self.W = nn.Linear(num_input, num_input)
#         self.U = nn.Linear(num_input, 1)

#     def forward(self, x):
#         if isPrint : print("SelfAttention 前 x.shape：", x.shape)
#         u = F.tanh(self.W(x))
#         if isPrint : print("--> F.tanh(self.W(x)) 后 u.shape：", u.shape, "self.W：", self.W)
#         a = F.softmax(self.U(u), dim=1)
#         if isPrint : print("--> F.softmax(self.U(u), dim=1) 后 a.shape：", a.shape, "self.U：", self.U)
#         res = torch.mul(a, x).sum(dim=1)
#         if isPrint : print("--> torch.mul(a, x).sum(dim=1) 后 res.shape：", res.shape)
#         return res


# class HAN(nn.Module):
#     def __init__(self, num_embeddings = 5845,
#                  num_classes = 10,
#                  num_words = 20,        # 每句话最多多少个词
#                  num_sentence = 10,     # 一篇文章多少个句子
#                  embedding_dim = 200,
#                  hidden_size_gru = 50,
#                  hidden_size_att = 100,
#                  ):
#         super(HAN, self).__init__()

#         self.num_words = num_words
#         self.num_sentence = num_sentence
#         self.embed = nn.Embedding(num_embeddings, embedding_dim, 0)

#         self.GRU1 = nn.GRU(embedding_dim,
#                            hidden_size_gru,
#                            bidirectional=True,  # 双向  Default: ``False``
#                            batch_first=True,    # : If ``True``, then the input and output tensors are provided as (batch, seq, feature). Default: ``False``
#                            )
#         self.self_attention1 = SelfAttention(hidden_size_gru * 2)


#         self.GRU2 = nn.GRU(hidden_size_gru * 2,
#                            hidden_size_gru * 2,
#                            bidirectional=True,  # 双向  Default: ``False``
#                            batch_first=True,    # : If ``True``, then the input and output tensors are provided as (batch, seq, feature). Default: ``False``
#                            )
#         self.self_attention2 = SelfAttention(hidden_size_gru * 4)

#         self.fc = nn.Linear(hidden_size_gru * 4, num_classes)

#     def forward(self, x:torch.Tensor, isSentenceSplit:bool=True):
#         if isPrint : print()
#         if isPrint : print()
#         if isPrint : print("x:", x.shape)
#         if isSentenceSplit:
#             sentences = []

#             for i in range(self.num_sentence):
#                 sentence = x[:, i * self.num_words: (i + 1) * self.num_words]
#                 if isPrint : print("-> sentence:", sentence.shape)
#                 sentence = self.embed(sentence)
#                 if isPrint : print("-> embed 后 sentence:", sentence.shape)
#                 sentence, _ = self.GRU1(sentence)
#                 if isPrint : print("-> GRU1 后 sentence:", sentence.shape)
#                 sentence = self.self_attention1(sentence)
#                 if isPrint : print("-> self_attention1 后 sentence:", sentence.shape)
#                 sentences.append(sentence)
#             sentences = torch.cat(sentences, dim=1)
#             if isPrint : print()
#             if isPrint : print("-> torch.cat(sentences, dim=1) 后 sentences:", sentences.shape)
#             x = sentences.view(sentences.size(0), self.num_sentence, -1)
#             if isPrint : print("-> sentences.view 后 x:", x.shape)
#         else:
#             sentences = self.embed(x)
#             if isPrint : print("-> embed 后 sentences:", sentences.shape)
#             sentences, _ = self.GRU1(sentences)
#             if isPrint : print("-> GRU1 后 sentences:", sentences.shape)
#             sentences = self.self_attention1(sentences)
#             if isPrint : print("-> self_attention1 后 sentences:", sentences.shape)
#             x = sentences
#         if isPrint : print()
#         if isPrint : print("view2 后 x:", x.shape)
#         x, _ = self.GRU2(x)
#         if isPrint : print("GRU2 后 x:", x.shape)
#         x = self.self_attention2(x)
#         if isPrint : print("self_attention2 后 x:", x.shape)
#         x = self.fc(x)
#         if isPrint : print("fc 后 x:", x.shape)
#         return F.softmax(x, dim=1)

In [0]:
class SelfAttention(nn.Module):
    def __init__(self, num_input, num_hidden):
        super(SelfAttention, self).__init__()
        self.W = nn.Linear(num_input, num_hidden)
        self.U = nn.Linear(num_hidden, 1)

    def forward(self, x):
        u = F.tanh(self.W(x))
        a = F.softmax(self.U(u), dim=1)
        return torch.mul(a, x).sum(dim=1)


class HAN(nn.Module):
    def __init__(self, num_embeddings = 5845,
                 num_classes = 10,
                 num_words = 100,
                 embedding_dim = 200,
                 hidden_size_gru = 50,
                 hidden_size_att = 100,
                 ):
        super(HAN, self).__init__()

        self.num_words = num_words
        self.embed = nn.Embedding(num_embeddings, embedding_dim, 0)

        self.GRU1 = nn.GRU(embedding_dim,
                           hidden_size_gru,
                           bidirectional=True, # 双向  Default: ``False``
                           batch_first=True, # : If ``True``, then the input and output tensors are provided as (batch, seq, feature). Default: ``False``
                           )
        self.self_attention1 = SelfAttention(hidden_size_gru * 2, hidden_size_att)


        self.GRU2 = nn.GRU(hidden_size_gru * 2,
                           hidden_size_gru,
                           bidirectional=True,  # 双向  Default: ``False``
                           batch_first=True,    # : If ``True``, then the input and output tensors are provided as (batch, seq, feature). Default: ``False``
                           )
        self.self_attention2 = SelfAttention(hidden_size_gru * 2, hidden_size_att)

        # self.fc = nn.Linear(hidden_size_att, num_classes)
        self.fc = nn.Linear(hidden_size_gru * 2, num_classes)

    def forward(self, x:torch.Tensor):
        # view() 说明：Returns a new tensor with the same data as the self tensor but of a different shape.
        # size() 说明：Returns the size of the self tensor. The returned value is a subclass of tuple.
        # contiguous() 说明： Returns a contiguous tensor containing the same data as self tensor. If self tensor is contiguous, this function returns the self tensor.

        '''
x: torch.Size([128, 100]) 100
view 后 x: torch.Size([12800, 1])
embed 后 x: torch.Size([12800, 1, 200])
GRU1 后 x: torch.Size([12800, 1, 400]) self.GRU1: GRU(200, 200, batch_first=True, bidirectional=True)
SelfAttention 前 x.shape： torch.Size([12800, 1, 400])
-> F.tanh(self.W(x)) 后 u.shape： torch.Size([12800, 1, 400]) self.W： Linear(in_features=400, out_features=400, bias=True)
-> F.softmax(self.U(u), dim=1) 后 a.shape： torch.Size([12800, 1, 1]) self.U： Linear(in_features=400, out_features=1, bias=True)
-> torch.mul(a, x).sum(dim=1) 后 res.shape： torch.Size([12800, 400])
self_attention1 后 x: torch.Size([12800, 400])
view2 后 x: torch.Size([128, 100, 400])
GRU2 后 x: torch.Size([128, 100, 400]) self.GRU2: GRU(400, 200, batch_first=True, bidirectional=True)
SelfAttention 前 x.shape： torch.Size([128, 100, 400])
-> F.tanh(self.W(x)) 后 u.shape： torch.Size([128, 100, 400]) self.W： Linear(in_features=400, out_features=400, bias=True)
-> F.softmax(self.U(u), dim=1) 后 a.shape： torch.Size([128, 100, 1]) self.U： Linear(in_features=400, out_features=1, bias=True)
-> torch.mul(a, x).sum(dim=1) 后 res.shape： torch.Size([128, 400])
self_attention2 后 x: torch.Size([128, 400])
fc 后 x: torch.Size([128, 3])
        '''
        # print("x:", x.shape, self.num_words)
        x = x.view(x.size(0) * self.num_words, -1).contiguous()
        # print("view 后 x:", x.shape)
        x = self.embed(x)
        # print("embed 后 x:", x.shape)
        x, _ = self.GRU1(x)
        # print("GRU1 后 x:", x.shape)
        # 根据打印结果显示，维度1的大小是1，self_attention1中计算后softmax最后得到了一堆1的值，
        # 然后此结果又和x相乘，最后得到的还是x，所以整个self_attention1相当于没有计算。所以这里不要这一行
        # x = self.self_attention1(x)  
        x = x.sum(1)
        # print("self_attention1 后 x:", x.shape)
        x = x.view(x.size(0) // self.num_words, self.num_words, -1)
        # print("view2 后 x:", x.shape)
        x, _ = self.GRU2(x)
        # print("GRU2 后 x:", x.shape)
        x = self.self_attention2(x)
        # print("self_attention2 后 x:", x.shape)
        x = self.fc(x)
        # print("fc 后 x:", x.shape)
        return F.softmax(x, dim=1)

In [23]:

#创建模型
wordEmbedding = torch.FloatTensor(wordEmbedding)
num_embeddings = len(word2vec.wv.index2word)
# Classical
# model = HAN(num_embeddings,
#             num_classes = 3,
#             embedding_dim = word2vec.wv.vector_size,
#             num_words = 10,
#             num_sentence = 10,
#             hidden_size_gru = 512,
#             hidden_size_att = 768,
#             )
model = HAN(num_embeddings,
            num_classes = 3,
            embedding_dim = word2vec.wv.vector_size,
            num_words = 100,
            hidden_size_gru = 300,
            hidden_size_att = 500,
            )
print(model)

modelParams = model.parameters()
for param in modelParams:
    if len(param.data.shape) > 1:
        # print('---', param.data.shape, param.data)
        torch.nn.init.kaiming_normal(param.data)
        # print('--->', param.data)
        
model.embed.from_pretrained(wordEmbedding)


##%% md

## 开始训练过程

##%%

def trainOneEpoch(epoch, model:HAN, trainLoader, optimizer:Optimizer, lossFunc):
    if torch.cuda.is_available():
        model = model.cuda()
        lossFunc = lossFunc.cuda()

    model.train()

    startTime = time.time()
    for i, (x, y) in enumerate(trainLoader):
        if torch.cuda.is_available():
            x = x.cuda()
            y = y.cuda()

        outputs = model(x)
        loss = lossFunc(outputs, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 100 == 99:
            print('Epoch %d, %d/%d, loss:%f ' % (epoch, i, len(trainLoader), loss))
        # if i > 2000:
        #     break 
    print('Epoch %d cost time: %.3fs' % (epoch, time.time() - startTime))


def testModel(epoch, model:HAN, testLoader):
    if torch.cuda.is_available():
        model = model.cuda()

    model.eval()

    total = 0
    correct = 0

    startTime = time.time()
    for i, (x, y) in enumerate(testLoader):
        if torch.cuda.is_available():
            x = x.cuda()
            y = y.cuda()

        outputs = model(x)
        _, predicted = torch.max(outputs.data, 1)

        total += len(y)
        correct += predicted.data.eq(y.data).cpu().sum().numpy()
        

        if i % 50 == 49:
            print('Epoch Test %d, %d/%d' % (epoch, i, len(testLoader)))
        # if i > 2000:
        #     break 
    print('Epoch Test %d cost time: %.3fs' % (epoch, time.time() - startTime))
    print('准确率： %.3f' % (correct / total))
    return correct / total


def train(epoch, model, modelSavePath, isLoad, lr=0.0002):
    last_acc = 0
    if isLoad: 
        model.load_state_dict(torch.load(modelSavePath))
        last_acc = testModel(epoch, model, testLoader)
    # optimizer=torch.optim.SGD(model.parameters(), momentum=0.9, lr=0.001)
    optimizer=torch.optim.Adam(model.parameters(), lr=0.0002)
    lossFunc =torch.nn.CrossEntropyLoss()
    for epoch in range(epoch):
        trainOneEpoch(epoch, model, trainLoader, optimizer, lossFunc)
        acc = testModel(epoch, model, testLoader)
        if last_acc < acc:
          torch.save(model.state_dict(), modelSavePath)
        last_acc = acc

train(20, model, 'data/EmotionAnalyzeModel_balance_TWV.model', False, lr=0.001)
    

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


HAN(
  (embed): Embedding(99999, 200, padding_idx=0)
  (GRU1): GRU(200, 300, batch_first=True, bidirectional=True)
  (self_attention1): SelfAttention(
    (W): Linear(in_features=600, out_features=500, bias=True)
    (U): Linear(in_features=500, out_features=1, bias=True)
  )
  (GRU2): GRU(600, 300, batch_first=True, bidirectional=True)
  (self_attention2): SelfAttention(
    (W): Linear(in_features=600, out_features=500, bias=True)
    (U): Linear(in_features=500, out_features=1, bias=True)
  )
  (fc): Linear(in_features=600, out_features=3, bias=True)
)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1558: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Epoch 0, 99/1036, loss:0.939127 
Epoch 0, 199/1036, loss:0.887987 
Epoch 0, 299/1036, loss:0.885514 
Epoch 0, 399/1036, loss:0.870773 
Epoch 0, 499/1036, loss:0.865416 
Epoch 0, 599/1036, loss:0.890627 
Epoch 0, 699/1036, loss:0.851046 
Epoch 0, 799/1036, loss:0.849178 
Epoch 0, 899/1036, loss:0.841529 
Epoch 0, 999/1036, loss:0.858986 
Epoch 0 cost time: 385.529s
Epoch Test 0, 49/259
Epoch Test 0, 99/259
Epoch Test 0, 149/259
Epoch Test 0, 199/259
Epoch Test 0, 249/259
Epoch Test 0 cost time: 29.171s
准确率： 0.688
Epoch 1, 99/1036, loss:0.832145 
Epoch 1, 199/1036, loss:0.857766 
Epoch 1, 299/1036, loss:0.839933 
Epoch 1, 399/1036, loss:0.809588 
Epoch 1, 499/1036, loss:0.858444 
Epoch 1, 599/1036, loss:0.830124 
Epoch 1, 699/1036, loss:0.799018 
Epoch 1, 799/1036, loss:0.851675 
Epoch 1, 899/1036, loss:0.820567 
Epoch 1, 999/1036, loss:0.813953 
Epoch 1 cost time: 384.086s
Epoch Test 1, 49/259
Epoch Test 1, 99/259
Epoch Test 1, 149/259
Epoch Test 1, 199/259
Epoch Test 1, 249/259
Epoch T

In [0]:
train(20, model, 'data/EmotionAnalyzeModel_balance_TWV.model', True, lr=0.0003)
train(20, model, 'data/EmotionAnalyzeModel_balance_TWV.model', True, lr=0.0001)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1558: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Epoch Test 20, 49/259
Epoch Test 20, 99/259
Epoch Test 20, 149/259
Epoch Test 20, 199/259
Epoch Test 20, 249/259
Epoch Test 20 cost time: 29.057s
准确率： 0.799
Epoch 0, 99/1036, loss:0.714727 
Epoch 0, 199/1036, loss:0.727061 
Epoch 0, 299/1036, loss:0.735342 
Epoch 0, 399/1036, loss:0.713944 
Epoch 0, 499/1036, loss:0.747210 
Epoch 0, 599/1036, loss:0.715627 
Epoch 0, 699/1036, loss:0.735560 
Epoch 0, 799/1036, loss:0.742908 
Epoch 0, 899/1036, loss:0.721923 
Epoch 0, 999/1036, loss:0.717881 
Epoch 0 cost time: 386.471s
Epoch Test 0, 49/259
Epoch Test 0, 99/259
Epoch Test 0, 149/259
Epoch Test 0, 199/259
Epoch Test 0, 249/259
Epoch Test 0 cost time: 29.250s
准确率： 0.800
Epoch 1, 99/1036, loss:0.714024 
Epoch 1, 199/1036, loss:0.742599 
Epoch 1, 299/1036, loss:0.707694 
Epoch 1, 399/1036, loss:0.715391 
Epoch 1, 499/1036, loss:0.707483 
Epoch 1, 599/1036, loss:0.757463 
Epoch 1, 699/1036, loss:0.722609 
Epoch 1, 799/1036, loss:0.747598 
Epoch 1, 899/1036, loss:0.741102 
Epoch 1, 999/1036, l

In [25]:

def eval(model, modelSavePath, isLoad = True):
    if isLoad: model.load_state_dict(torch.load(modelSavePath))
    testModel(0, model, testLoader)

eval(model, 'EmotionAnalyzeModelData_ClassicalHAN_OB_plus.model', isLoad = True)

RuntimeError: ignored

In [0]:
lossFunc =torch.nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
_, predicted = torch.max(input.data, 1)
output = lossFunc(input, target)
print('input',input, '\n target',target, '\n output', output)
output = lossFunc(input, predicted)
print('predicted',predicted, '\n target',target, '\n output', output)

print(predicted.data.eq(target).cpu().sum())
print(target.data.eq(predicted).cpu().sum())

In [0]:
# 何凯明初始化

w = torch.Tensor(3, 5, 2)
print(w)
print(nn.init.kaiming_uniform(w))
print(w)
w = torch.Tensor(3, 5, 2)
print(w)
print(torch.nn.init.kaiming_normal(w))
print(w)